## Gradient Descent Code-Along

Let's walk through how gradient descent works using code.

In [7]:
import pandas as pd
import numpy as np

In [8]:
np.random.seed(42)

In [9]:
temp = np.random.uniform(-10, 80, 100) # temperature in degrees Fahrenheit

In [10]:
print(np.mean(temp))
print(np.std(temp, ddof = 1))

32.31626690403884
26.77404699137873


In [11]:
# Beers sold at a football stadium(AMERICAN!lol)

In [12]:
beers_sold = 200000 + 1000 * temp + np.random.normal(0, 20000)

In [7]:
beers_sold

array([225449.55206103, 277305.22894166, 257620.39612779, 245620.2049425 ,
       205782.61900458, 205780.44819502, 196968.4664599 , 269696.79448451,
       245841.29242165, 255467.47336641, 193593.54585139, 279032.82805934,
       266660.7790368 , 210851.46132581, 208105.1884134 , 208247.34725157,
       219122.74323112, 238969.02021166, 230615.99304255, 217951.56398259,
       246807.70188978, 204295.38882345, 218033.95973293, 224713.5072612 ,
       232787.2399443 , 262406.77789013, 209711.58175902, 238022.04082199,
       245058.25256235, 195921.47850956, 246419.97803589, 207088.11249662,
       197595.58473344, 277140.63971756, 278647.82434147, 264496.70269524,
       219156.18059037, 200531.43162534, 253321.91375086, 231354.66580133,
       202724.38250079, 236306.86327478, 194835.90826513, 273579.77755185,
       215031.13970876, 251367.94695662, 219794.93821281, 238547.06327077,
       240944.86650566, 208377.84236206, 279003.55786357, 261502.89546726,
       276295.84610554, 2

$$ \text{beers_sold}_i = 200000 + 1000 * \text{temp}_i + \varepsilon_i $$

In [8]:
df = pd.DataFrame({'temp': temp,
                   'beers_sold': beers_sold})

In [9]:
df.head()

beers_sold       temp
0  225449.552061  23.708611
1  277305.228942  75.564288
2  257620.396128  55.879455
3  245620.204942  43.879264
4  205782.619005   4.041678

#### Our goal is to fit a model here.
- You and I know that our $y$-intercept $\beta_0$ is 200,000.
- You and I know that our slope $\beta_1$ is 1,000.
- However, our computer does not know that. Our computer has to estimate $\hat{\beta}_0$ and $\hat{\beta}_1$ from the data.
    - We might say that our **machine** has to... **learn**.

#### Our workflow:
1. Instantiate model.
2. Select a learning rate $\alpha$.
3. Select a starting point $\hat{\beta}_{1,0}$.
4. Calculate the gradient of the loss function.
5. Calculate $\hat{\beta}_{1,i+1} = \hat{\beta}_{1,i} - \alpha * \frac{\partial L}{\partial \beta_1}$.
6. Check value of $\left|\hat{\beta}_{1,i+1} - \hat{\beta}_{1,i}\right|$.
7. Repeat steps 4 through 6 until "stopping condition" is met.

#### Step 1. Instantiate model.

Our model takes on the form:
$$ Y = \beta_0 + \beta_1 X + \varepsilon$$

#### Step 2. Select a learning rate $\alpha$.

$$\alpha = 0.1$$

In [50]:
alpha = 0.01

#### Step 3. Select a starting point.
The zero-th iteration of $\hat{\beta}_1$ is going to start at, say, 20.
$$\hat{\beta}_{1,0} = 20$$

Two points:
- You and I know that the true value of $\beta_1$ is 1000. We need the computer to figure that part out!

#### Step 4. Calculate the gradient of the loss function with respect to parameter $\beta_1$.

The loss function, $L$, is our mean square error.

$$L = \sum_{i = 1} ^ n (y_i - \hat{y}_i)^2 $$

$$\Rightarrow L = \sum_{i = 1} ^ n (y_i - (\hat{\beta}_0 + \hat{\beta}_1x_i))^2 $$

The gradient of this loss function with respect to $\beta_1$ is:

$$\frac{\partial L}{\partial \beta_1} = 2 \sum_{i=1}^n -x_i(y_i - (\hat{\beta}_1x_i + \hat{\beta}_0)) $$

The gradient of this loss function with respect to $\beta_0$ is:
 
$$\frac{\partial L}{\partial \beta_0} =  2 \sum_{i=1}^n -(y_i - (\hat{\beta}_1x_i + \hat{\beta}_0)) $$

In [40]:
temp.shape

(100,)

In [42]:
beers_sold.shape

(100,)

In [77]:
def gradient_descent(alpha, x, y, epsilon = 0.001, max_iter = 10000):
    # setting converged = False 
    converged = False
    # starting at 0 iterations
    iter = 0
    # getting the size of the data
    n = x.shape[0]
    
    #### Step 3. Select a starting point
    beta_0 = 0
    beta_1 = 0
    
    # defining the error function, J
    J = np.sqrt(np.sum([(y[i] - (beta_0 + beta_1 * x[i])) ** 2 for i in range(n)]) )
    
    # Iterate Loop
    while not converged:
        #### Step 4. Calculate the gradient w.r.t the loss function
        grad0 = -2/n * np.sum([(y[i] - (beta_1 * x[i] + beta_0)) for i in range(n)])
        grad1 = -2/n * np.sum([x[i] * (y[i] - (beta_1 * x[i] + beta_0)) for i in range(n)])
        
        #### Step 5. update the weights
       
        beta_0 = beta_0 - alpha * grad0
        beta_1 = beta_1 - alpha * grad1
        
        #### Step 6. Check the stopping condition, in this case, we are checking that the 
        #### error in our model has been reduced to a low level, epsilon(0.001)
        e = np.sqrt(np.sum([(y[i] - (beta_0 + beta_1 * x[i])) ** 2 for i in range(n)]))
        
        if abs(J-e) <= epsilon:
            print ('Converged, iterations: ', iter, '!!!')
            converged = True
        print(abs(J-e))
        J = e   # update error 
        iter += 1  # update iter
    
        if iter == max_iter:
            print ('Max iterations exceeded!')
            converged = True
       
    return beta_0,beta_1
        

In [79]:
gradient_descent(alpha = 0.00015, x = temp, y = beers_sold, epsilon = 0.01, max_iter =100000)

768047.3971080454
230831.659152325
57515.64180406998
13374.860283136833
3139.855290960055
825.9899812822696
305.98157040425576
189.26351583423093
163.06227215728723
157.16958547057584
155.83299344801344
155.5185142941773
155.4333360379096
155.3996004955843
155.37740728794597
155.35780524043366
155.33878626325168
155.31989986542612
155.30104498332366
155.2821989459917
155.26335666584782
155.24451700178906
155.22567969863303
155.2068446970079
155.18801198457368
155.16918155830353
155.15035341726616
155.13152756053023
155.1127039885614
155.0938826999627
155.07506369566545
155.05624697473831
155.03743253694847
155.01862038229592
154.99981050984934
154.98100292030722
154.9621976125054
154.94339458667673
154.92459384235553
154.90579537954181
154.88699919730425
154.86820529587567
154.84941367502324
154.83062433474697
154.81183727341704
154.79305249243043
154.774269990623
154.75548976752907
154.7367118236143
154.71793615841307
154.69916277052835
154.68039166158997
154.66162282973528
154.642856

139.5550173157826
139.53808378847316
139.52115231589414
139.50422289804555
139.48729553376324
139.47037022421136
139.4534469677601
139.4365257648751
139.4196066155564
139.40268951887265
139.38577447505668
139.36886148341
139.35195054416545
139.33504165732302
139.31813482148573
139.30123003735207
139.28432730445638
139.26742662279867
139.2505279914476
139.23363141086884
139.2167368803639
139.19984439969994
139.1829539691098
139.166065587895
139.14917925558984
139.1322949724272
139.1154127381742
139.0985325523652
139.0816544147674
139.0647783249151
139.04790428304113
139.03103228867985
139.01416234113276
138.99729444109835
138.98042858787812
138.96356478054076
138.94670301978476
138.92984330514446
138.9129856361542
138.89613001258112
138.87927643442526
138.86242490122095
138.8455754125025
138.82872796873562
138.81188256898895
138.7950392132625
138.77819790132344
138.76135863293894
138.74452140764333
138.72768622543663
138.71085308631882
138.69402198912576
138.6771929350216
138.6603659221

128.01933900127187
128.00380520685576
127.98827329743654
127.97274327208288
127.95721513149329
127.94168887520209
127.92616450227797
127.91064201388508
127.89512140816078
127.87960268650204
127.86408584727906
127.84857089119032
127.8330578173045
127.81754662608728
127.80203731707297
127.7865298895631
127.77102434425615
127.75552067975514
127.74001889699139
127.72451899480075
127.7090209738817
127.69352483260445
127.6780305725988
127.6625381920021
127.64704769151285
127.63155907066539
127.61607232899405
127.60058746649884
127.58510448341258
127.56962337833829
127.55414415197447
127.53866680408828
127.52319133398123
127.50771774188615
127.49224602687173
127.47677618963644
127.4613082294818
127.44584214570932
127.43037793925032
127.41491560870782
127.39945515454747
127.38399657653645
127.36853987397626
127.35308504709974
127.33763209520839
127.32218101876788
127.30673181707971
127.29128448991105
127.27583903679624
127.26039545820095
127.24495375377592
127.229513922357
127.2140759647591
12

119.13087591447402
119.11642064107582
119.10196712147444
119.08751535590272
119.07306534366217
119.0586170849856
119.04417057952378
119.02972582681105
119.0152828268474
119.0008415794
118.98640208423603
118.97196434123907
118.95752835005987
118.9430941103492
118.92866162245627
118.91423088533338
118.89980189956259
118.88537466456182
118.87094918033108
118.85652544593904
118.8421034620842
118.82768322818447
118.8132647437742
118.79884800943546
118.78443302388769
118.77001978771295
118.75560830021277
118.74119856150355
118.7267905715853
118.71238432941027
118.69797983556055
118.68357708945405
118.66917609097436
118.65477684000507
118.64037933619693
118.62598357920069
118.61158956913278
118.59719730576035
118.58280678850133
118.56841801747214
118.55403099220712
118.53964571293909
118.52526217908598
118.51088039029855
118.49650034680963
118.48212204826996
118.46774549421389
118.453370684525
118.43899761931971
118.42462629801594
118.41025672049727
118.39588888641447
118.38152279611677
118.3

108.33281861490104
108.31967357138637
108.30653012322728
108.29338826995809
108.28024801088031
108.26710934680887
108.25397227657959
108.24083680042531
108.22770291834604
108.21457062964328
108.20143993454985
108.18831083283294
108.17518332402688
108.16205740824807
108.14893308491446
108.13581035414245
108.12268921593204
108.10956966970116
108.09645171533339
108.08333535282873
108.0702205816051
108.05710740177892
108.0439958133502
108.03088581562042
108.01777740858961
108.004670592607
107.99156536627561
107.97846173075959
107.96535968501121
107.95225922879763
107.93916036258452
107.92606308555696
107.91296739771497
107.89987329929136
107.88678078912199
107.87368986790534
107.86060053505935
107.847512790584
107.83442663389724
107.8213420654647
107.80825908435509
107.79517769091763
107.78209788468666
107.76901966577861
107.75594303326216
107.7428679879522
107.72979452891741
107.71672265627421
107.70365237002261
107.69058366911486
107.67751655448228
107.66445102519356
107.65138708124869
1

...

0.015086124454157357
0.015084293915663238
0.015082463607328123
0.015080633441058922
0.015078803700532717
0.015076973916521297
0.015075144515108718
0.015073315289868106
0.015071486263067868
0.01506965761267054
0.015067828935059424
0.01506600065965813
0.015064172542167853
0.015062344805571115
0.01506051708216205
0.015058689692992289
0.015056862361419121
0.015055035326255961
0.015053208753968761
0.015051382089254162
0.015049555824717231
0.015047729787724506
0.01504590388239535
0.015044078189774268
0.015042252661345401
0.01504042756613444
0.015038602557325476
0.015036777804937174
0.015034953211113589
0.015033128811140273
0.015031304768328368
0.015029480740395229
0.015027657080693757
0.015025833794425125
0.015024010481610617
0.015022187387927488
0.015020364644755091
0.015018542119520362
0.015016719724684435
0.015014897724825005
0.015013075748555593
0.015011254122114792
0.015009432652377086
0.015007611425801315
0.015005790320586243
0.015003969647182203
0.015002148947715455
0.0150003285824453

0.013362183731089772
0.013360562426115052
0.013358941288572623
0.013357320337405554
0.013355699563888379
0.013354078999341823
0.01335245858697931
0.013350838367898632
0.013349218357177506
0.013347598702495134
0.013345979040366274
0.013344359571604514
0.013342740484503679
0.013341121367162145
0.013339502633570532
0.013337884118868715
0.013336265561378013
0.01333464736796941
0.013333029393450602
0.013331411586761988
0.013329793959300673
0.013328176549620707
0.013326559284365658
0.013324942215618307
0.013323325343890247
0.013321708809186816
0.013320092344997647
0.01331847600039282
0.013316860048831813
0.013315244090733813
0.013313628507091835
0.013312012915989158
0.013310397726201018
0.013308782704513078
0.013307167830433286
0.013305553025730887
0.013303938527585046
0.013302324240243024
0.013300710140342176
0.013299096241382813
0.013297482667894656
0.01329586914366132
0.01329425576497556
0.013292642599139981
0.01329102979423169
0.013289416977428914
0.013287804553613114
0.01328619211665227

0.01183667965766233
0.011835243438298448
0.011833807423442977
0.011832371414499221
0.011830935797334519
0.011829500118864189
0.011828064864516818
0.011826629609743122
0.011825194507380843
0.01182375961411708
0.01182232488567081
0.011820890535744866
0.011819456179580357
0.011818022057966004
0.011816587884482033
0.011815154064279909
0.011813720493577762
0.011812287058219795
0.011810853820676925
0.011809420621474942
0.01180798768682223
0.011806554869622232
0.011805122370049048
0.011803689868685296
0.011802257546591477
0.01180082563736562
0.011799393674934322
0.011797961945632096
0.011796530379555747
0.011795098950798888
0.01179366780021951
0.011792236761820618
0.01179080573379565
0.011789375125161428
0.011787944657001503
0.011786514364189316
0.011785084105767396
0.011783654201792615
0.011782224290513454
0.011780794581241594
0.011779365261091357
0.011777935873183765
0.011776506722142699
0.011775077770209919
0.011773648908359746
0.011772220358409413
0.011770791955655113
0.011769363676350508

0.010484063656235776
0.010482791623786625
0.010481519645580306
0.010480247840391144
0.01047897601988268
0.010477704618693906
0.010476433159496423
0.010475162085455736
0.010473891032944493
0.010472620136752653
0.010471349277196396
0.010470078747019329
0.010468808246471895
0.010467538095369378
0.010466267966677378
0.010464998019841687
0.010463728055100319
0.010462458469476132
0.010461188876391248
0.010459919688585728
0.010458650450644313
0.01045738142005348
0.01045611241946176
0.01045484374407124
0.010453575069874432
0.010452306600839734
0.01045103849003226
0.010449770224269628
0.010448502276972249
0.010447234552827922
0.01044596684306498
0.010444699280256486
0.010443431904704426
0.010442164739600912
0.010440897732678422
0.010439630769326413
0.010438364119266907
0.010437097487951519
0.01043583110104862
0.010434564798103452
0.010433298738959706
0.010432032681819692
0.010430766985237483
0.010429501294325405
0.01042823576422336
0.010426970458965457
0.010425705107877548
0.010424440163134818


(201727.98724864452, 1000.2387228783595)

#### Step 5. Calculate $\hat{\beta}_{1,i+1} = \hat{\beta}_{1,i} - \alpha * \frac{\partial L}{\partial \beta_1}$.

#### Step 6. Check value of $\left|\hat{\beta}_{1,i+1} - \hat{\beta}_{1,i}\right|$.

#### Step 7: Save final value of $\hat{\beta}_1$.